In [1]:
import matplotlib.pyplot as plt
from pyvis.network import Network
import networkx as nx
from pathlib import Path
import pandas as pd
import numpy as np

import nest_asyncio

from llama_index.core import Response
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    DatasetGenerator,
    BatchEvalRunner,
    RelevancyEvaluator,
    CorrectnessEvaluator,
)

from notebooks.helpers.bot.promtps import question_gen_query
from evaluate_pairings import prepare_evalution_qa, parse_evalutions
from app.notebooks.helpers.bot.kg_generation import create_kg_triplets
from notebooks.helpers.bot.bot import (
    get_chat_engine,
    get_query_engine,
    get_simple_query,
    load_llm,
    load_embedding_model,
    setup_index_and_storage,
    generate_pairings_documents,
    service,
)

nest_asyncio.apply()
%matplotlib inline

In [2]:
def display_eval_df(query: str, response: Response, eval_result: Response) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response),
            "Source": (' '.join(eval_result.contexts)[:1000] + "..."),
            "Evaluation Result": eval_result.passing,
        },
        index = [0]
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)



In [ ]:
def parse_triplets(text):
    triplets = []
    rows = text.split('\n')
    for row in rows[:-1]:
        triplet = row.split('**')
        triplets.append((triplet[0], triplet[1], triplet[2]))
    return triplets

kg_triplets = []

KG = create_kg_triplets(sample_size=10, for_model=False)
G = nx.DiGraph()
for _, row in KG.iterrows():
    triplets = parse_triplets(row['triplets'])
    for triplet in triplets:
        G.add_edge(triplet[0], triplet[2], label=triplet[1])

In [3]:
KG = create_kg_triplets(sample_size=601)
kg_pairings = KG.apply(generate_pairings_documents, axis=1)

In [4]:
print(kg_pairings.iloc[0].get_content(metadata_mode='llm'))
print(kg_pairings.iloc[0].get_node_info())

Metadata: category=>Red
variety_location=>Pinot Noir Victoria, Australia
alcohol=>13.5%
price=>$42
variety=>Pinot Noir
vintage=>2014
winery=>Giant Steps
country=>Australia
province=>Victoria
region=>Yarra Valley
-----
Content: The object wine, **Pinot Noir** has a relationship of **type_of** with the object **Red
The object wine, **Pinot Noir** has a relationship of **variety_location** with the object **Pinot Noir Victoria, Australia
The object wine, **Giant Steps 2014 Applejack Vineyard Pinot Noir (Yarra Valley)** has a relationship of **has_alcohol** with the object **13.5%
The object wine, **Giant Steps 2014 Applejack Vineyard Pinot Noir (Yarra Valley)** has a relationship of **has_price_of** with the object **$42
The object wine, **Giant Steps 2014 Applejack Vineyard Pinot Noir (Yarra Valley)** has a relationship of **has_variety** with the object **Pinot Noir
The object wine, **Giant Steps 2014 Applejack Vineyard Pinot Noir (Yarra Valley)** has a relationship of **vintage_year_of

In [5]:
llm = load_llm('openai3.5')

In [6]:
embed_model = load_embedding_model("openai3")
service_context = service(llm=llm, embed_model=embed_model)

In [7]:
storage_context, kg_index = setup_index_and_storage(
        service=service_context,
        kg_pairings=kg_pairings,
        show_progress=False,
        max_triplets_chunk=10,
        force=False,
    )

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Upgrading previously saved KG index to new storage format.


In [8]:
query_engine = get_query_engine(
    kg_index,
    chat_mode="simple",
    retriver_mode="hybrid",
    response_mode="compact",
    use_global_node_triplets=True,
    max_keywords_per_query=10,
    num_chunks_per_query=10,
    similarity_top_k=3,
    graph_store_query_depth=3,
    include_text = True
)

In [9]:
eval_dataset = prepare_evalution_qa()
eval_dataset = eval_dataset.sample(n=3, random_state=23)

In [ ]:
relevancy_eval = RelevancyEvaluator(service_context=service_context)
faithfulness_eval = FaithfulnessEvaluator(service_context=service_context)
correctness_eval = CorrectnessEvaluator(service_context=service_context)

In [ ]:
eval_dataset["questions"].to_list()



In [ ]:
runner = BatchEvalRunner(
    {
        # "faithfulness": faithfulness_eval,
        "relevancy": relevancy_eval,
        },
    workers=8,
    show_progress=True
)

references = eval_dataset["responses"].to_list()

eval_results = runner.evaluate_queries(
    query_engine,
    queries=eval_dataset["questions"].to_list(),
    # eval_kwargs_lists={
        # 'relevancy': {'', []},
        # 'correctness': {'reference': references },
    # },
)

In [ ]:
eval_dataset["responses"].to_list()[0]

In [ ]:
print(eval_results.keys())
print(eval_results["faithfulness"][0])

In [ ]:
from evaluate_pairings import parse_evalutions

In [ ]:
parse_evalutions(
    eval_results=eval_results,
    model="gpt-3.5",
    embedding_model="gpt-3.5",
    response_mode="compact",
    retriever_mode="hybrid",
    chat_mode="simple",  # Irrelevant if the response are from QueryEngine
    evaluation_dataset=eval_dataset,
)

In [ ]:
print(eval_results["faithfulness"][1].passing)
print(eval_results["faithfulness"][1].response)
print(eval_results["faithfulness"][1].contexts)
print(eval_results["faithfulness"][1].score)
print(eval_results["faithfulness"][1].pairwise_source)
print(eval_results["faithfulness"][1].invalid_result)
print(eval_results["faithfulness"][1].invalid_reason)

In [ ]:
chat_engine = get_chat_engine(
    kg_index,
    chat_mode="context",
    retriver_mode="hybrid",
    use_global_node_triplets=True,
    max_keywords_per_query=10,
    num_chunks_per_query=10,
    similarity_top_k=3,
    graph_store_query_depth=3,
)

response = chat_engine.chat("What wine would you recommend pairing with a grilled steak?")
print(response)

In [ ]:
display_eval_df(question, response, eval_result)

In [ ]:
G = kg_index.get_networkx_graph(limit=1000)

In [ ]:
net = Network(notebook = True, cdn_resources = "remote",
                # bgcolor = "#222222",
                font_color = "black",
                height = "750px",
                width = "100%",
                select_menu = True,
                filter_menu = True,
)
net.show_buttons(filter_="physics")
net.from_nx(G)
net.show("nx.html")

In [ ]:
pos = nx.spring_layout(G, seed=42, k=1.5)
labels = nx.get_edge_attributes(G, 'label')
plt.figure(figsize=(20, 20))
nx.draw(G, pos, font_size=8, node_size=200, node_color='lightblue', edge_color='gray', alpha=0.6)
# nx.draw_networkx_edge_labels(G, pos, font_size=3, label_pos=0.3, verticalalignment='baseline')
plt.title('Knowledge Graph')
plt.show()

In [ ]:
for edge in G.edges(data=True):
    print(edge)

In [ ]:
print(G.number_of_nodes())